# Los glaciares como recursos hídricos: parte 2 (proyecciones)

Objetivos de este cuaderno:
- ejecutar simulaciones utilizando proyecciones climáticas para explorar el papel de los glaciares como recursos hídricos

## Preparando el escenario: la escorrentía de los glaciares y el "pico de agua"

¡Recomendamos enfáticamente ejecutar [Part 1](glacier_water_resources.ipynb) de este cuaderno antes de continuar!

## Configuración

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_context('notebook')

In [ ]:
# One interactive plot below requires Bokeh
# The rest of the notebook works without this dependency
import holoviews as hv
hv.extension('bokeh')
import geoviews as gv
import geoviews.tile_sources as gts

In [ ]:
# Temporarily ignore warnings from shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning)

In [ ]:
import xarray as xr
import salem
import numpy as np
import pandas as pd

In [ ]:
import oggm.cfg
from oggm import utils, workflow, tasks, graphics

In [ ]:
# OGGM options
oggm.cfg.initialize(logging_level='WARNING')
oggm.cfg.PATHS['working_dir'] = utils.gettempdir(dirname='WaterResources')
oggm.cfg.PARAMS['min_ice_thick_for_length'] = 1  # a glacier is when ice thicker than 1m
oggm.cfg.PARAMS['store_model_geometry'] = True

### Definir el glaciar con el que jugaremos

Para este cuaderno usamos Hintereisferner, Austria. Algunas otras posibilidades para jugar:
-Hintereisferner, Austria: RGI60-11.00897
- Artesonraju, Peru: RGI60-16.02444
-Rikha Samba, Nepal: RGI60-15.04847
-Parlung No. 94, China: RGI60-15.11693
Y prácticamente cualquier glaciar en el que pueda encontrar el RGI Id, p. en el [GLIMS viewer](https://www.glims.org/maps/glims)! Sin embargo, los grandes glaciares pueden necesitar simulaciones más largas para ver los cambios. Para parámetros de calibración menos inciertos, también recomendamos elegir uno de los muchos glaciares de referencia [GLIMS viewer](https://www.glims.org/maps/glims), donde nos aseguramos de que las observaciones del balance de masa coincidan mejor.
Comencemos primero con Hintereisferner y al final de este cuaderno se le invitará a probar su glaciar favorito.

In [ ]:
# Hintereisferner
rgi_id = 'RGI60-11.00897'

## Preparando los datos del glaciar

Esto puede tardar unos minutos en la primera llamada debido a la descarga de los datos necesarios:

In [ ]:
# We pick the elevation-bands glaciers because they run a bit faster - but they create more step changes in the area outputs
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/elev_bands/qc3/pcp2.5/no_match'
gdir = workflow.init_glacier_directories([rgi_id], from_prepro_level=5, prepro_border=80, prepro_base_url=base_url)[0]

## Mapa glaciar interactivo

Un primer vistazo al glaciar de interés.
*Sugerencia: puede usar el mouse para desplazarse y hacer zoom en el mapa*

In [ ]:
sh = salem.transform_geopandas(gdir.read_shapefile('outlines'))
(gv.Polygons(sh).opts(fill_color=None, color_index=None) *
 gts.tile_sources['EsriImagery'] * gts.tile_sources['StamenLabels']).opts(width=800, height=500, active_tools=['pan', 'wheel_zoom'])

Para OGGM, los glaciares tienen una dimensión de "1,5" a lo largo de su línea de flujo:

In [ ]:
fls = gdir.read_pickle('model_flowlines')
graphics.plot_modeloutput_section(fls);

## "Ejecución de compromiso"

Ahora estamos listos para ejecutar nuestra primera simulación. Esta es la llamada "ejecución de compromiso": ¿cuánta pérdida de hielo está "ya comprometida" para este glaciar, incluso si el cambio climático se detuviera hoy? Este es un experimento útil pero puramente teórico: el cambio climático no se detendrá hoy, desafortunadamente. Para obtener más información sobre la pérdida de masa comprometida a escala global, lea [Marzeion et al., 2018](https://www.nature.com/articles/s41558-018-0093-1).
Aquí, ejecutamos una simulación durante 100 años con un clima constante basado en los últimos 11 años:

In [ ]:
# file identifier where the model output is saved
file_id = '_ct'

# We are using the task run_with_hydro to store hydrological outputs along with the usual glaciological outputs
tasks.run_with_hydro(gdir,
                     run_task=tasks.run_constant_climate,  # which climate? See below for other examples
                     nyears=100,  # length of the simulation
                     y0=2014, halfsize=5,  # For the constant climate, period over which the climate is taken from
                     store_monthly_hydro=True,  # Monthly ouptuts provide additional information
                     output_filesuffix=file_id);  # an identifier for the output file to read it later

Entonces podemos echar un vistazo a la salida:

In [ ]:
with xr.open_dataset(gdir.get_filepath('model_diagnostics', filesuffix=file_id)) as ds:
    # The last step of hydrological output is NaN (we can't compute it for this year)
    ds = ds.isel(time=slice(0, -1)).load()

¡Hay muchas variables en este conjunto de datos! Podemos enumerarlos con:

In [ ]:
ds

*Consejo: puede hacer clic en una variable y mostrar su atributo con el botón "página" a la derecha.*
Las variables `time` y `month_2d` son coordenadas, y las otras variables se proporcionan como información adicional (por ejemplo, `calendar_month`, volveremos a esto), o proporcionan los datos reales. Por ejemplo, podemos trazar la evolución anual del volumen y longitud de nuestro glaciar:

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(10, 7), sharex=True)
ds.volume_m3.plot(ax=axs[0]);
ds.length_m.plot(ax=axs[1]);
axs[0].set_xlabel(''); axs[0].set_title(f'{rgi_id}'); axs[1].set_xlabel('Years');

La longitud y el volumen del glaciar disminuyen durante los primeros ~40 años de la simulación: esta es la fase de **retroceso del glaciar**. Posteriormente, tanto la longitud como el volumen oscilan en torno a un valor más o menos constante indicando que el glaciar ha **alcanzado el equilibrio**. La diferencia entre el volumen inicial y el volumen de equilibrio se denomina **pérdida de masa comprometida**. Puede ser bastante alto en los Alpes y depende de muchos factores (como el tamaño del glaciar, la ubicación y el período climático de referencia),

### escurrimiento anual

A medida que los glaciares retroceden, contribuyen al aumento del nivel del mar (¡visite la aplicación [World Glaciers Explorer](https://edu.oggm.org/en/latest/explorer.html) OGGM-Edu para obtener más información!). Esto no es lo que nos interesa aquí. De hecho, también tendrán impactos locales importantes: en este cuaderno, veremos su impacto en el flujo de agua.
Echemos un vistazo a algunos de los resultados hidrológicos calculados por OGGM. Comenzamos creando un DataFrame de pandas de todas las variables "1D" (anuales):

In [ ]:
sel_vars = [v for v in ds.variables if 'month_2d' not in ds[v].dims]
df_annual = ds[sel_vars].to_dataframe()

Luego podemos seleccionar las variables hidrológicas y sumarlas para obtener la escorrentía total anual:

In [ ]:
# Select only the runoff variables
runoff_vars = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
# Convert them to megatonnes (instead of kg)
df_runoff = df_annual[runoff_vars] * 1e-9
fig, ax = plt.subplots(figsize=(10, 3.5), sharex=True)
df_runoff.sum(axis=1).plot(ax=ax);
plt.ylabel('Mt'); plt.xlabel('Years'); plt.title(f'Total annual runoff for {rgi_id}');

Las variables hidrológicas se calculan sobre la mayor área posible cubierta por hielo glaciar durante la simulación. Esto es equivalente a la escorrentía que se mediría en una estación hidrológica de calibre fijo en el extremo del glaciar.
La escorrentía total anual consta de los siguientes componentes:
- fusión fuera del glaciar: la nieve se derrite en áreas que ahora están libres de glaciares (es decir, 0 en el año de mayor extensión del glaciar, en este ejemplo al comienzo de la simulación)
- derretimiento en el glaciar: hielo + nieve estacional derretida en el glaciar
- precipitación líquida dentro y fuera del glaciar (esta última siendo cero en el año de mayor extensión glacial, en este ejemplo al comienzo de la simulación)

In [ ]:
f, ax = plt.subplots(figsize=(10, 6));
df_runoff.plot.area(ax=ax, color=sns.color_palette("rocket")); plt.xlabel('Years'); plt.ylabel('Runoff (Mt)'); plt.title(rgi_id);

Antes de continuar, recordemos la contribución esperada de los glaciares a la escorrentía.
<img src="https://raw.githubusercontent.com/OGGM/oggm-edu/master/docs/_static/fig_huss_hock.jpg" width="700px" alt="Figura 1 de https://www.nature. com/articulos/s41558-017-0049-x">
*Gráfico de [Huss & Hock (2018)](https://www.nature.com/articles/s41558-017-0049-x)*
**Preguntas para explorar:**
- **¿Dónde aproximadamente en este gráfico se encuentra el glaciar estudiado?**
- **¿Puedes explicar la contribución relativa de cada componente, con base en el cuaderno anterior?**

La escorrentía total de la cuenca de un glaciar es la suma de las cuatro contribuciones anteriores. Para mostrar que la contribución total del glaciar es de hecho cero ($\Delta M = 0$) **cuando está en equilibrio**, podemos calcularlo a partir del cambio de masa del glaciar:

In [ ]:
glacier_mass = ds.volume_m3.to_series() * oggm.cfg.PARAMS['ice_density'] * 1e-9  # In Megatonnes, Mt

glacier_mass.diff().plot()
plt.axhline(y=0, color='k', ls=':')
plt.ylabel('Annual glacier mass change (Mt yr$^{-1}$)')
plt.xlabel('Years'); plt.title('Glacier contribution to annual runoff');

¡Tenga en cuenta que esto no significa que el hielo no se esté derritiendo! En equilibrio, esto significa que el hielo que se derrite cada año sobre el glaciar es reemplazado por nieve en el área de acumulación del glaciar. Esto ilustra bien que **los glaciares en equilibrio no son recursos hídricos netos en promedio anual**: en el transcurso del año ganan tanta masa como liberan.

### escorrentía mensual

Las variables "2D" contienen los mismos datos pero a resolución mensual, con la dimensión (tiempo, mes). Por ejemplo, la escorrentía se puede calcular
mismo camino:

In [ ]:
# Select only the runoff variables and convert them to megatonnes (instead of kg)
monthly_runoff = (ds['melt_off_glacier_monthly'] + ds['melt_on_glacier_monthly'] +
                  ds['liq_prcp_off_glacier_monthly'] + ds['liq_prcp_on_glacier_monthly'])
monthly_runoff *= 1e-9
monthly_runoff.clip(0).plot(cmap='Blues', cbar_kwargs={'label': 'Mt'}); plt.xlabel('Months'); plt.ylabel('Years');

Pero tenga en cuenta que algo está un poco mal con esto: las coordenadas son meses hidrológicos; mejoremos esto:

In [ ]:
# This should work in both hemispheres maybe?
ds_roll = ds.roll(month_2d=ds['calendar_month_2d'].data[0] - 1, roll_coords=True)
ds_roll['month_2d'] = ds_roll['calendar_month_2d']

# Select only the runoff variables and convert them to megatonnes (instead of kg)
monthly_runoff = (ds_roll['melt_off_glacier_monthly'] + ds_roll['melt_on_glacier_monthly'] +
                  ds_roll['liq_prcp_off_glacier_monthly'] + ds_roll['liq_prcp_on_glacier_monthly'])
monthly_runoff *= 1e-9
monthly_runoff.clip(0).plot(cmap='Blues', cbar_kwargs={'label': 'Mt'}); plt.xlabel('Months'); plt.ylabel('Years');

Como podemos ver, la escorrentía es aproximadamente cero durante los meses de invierno, mientras que es relativamente alta durante los meses de verano. Esto implica que el glaciar es una fuente de agua en el verano cuando libera el agua acumulada en el invierno.
El ciclo anual cambia a medida que el glaciar retrocede:

In [ ]:
monthly_runoff.sel(time=[0, 30, 99]).plot(hue='time');
plt.title('Annual cycle');
plt.xlabel('Month');
plt.ylabel('Runoff (Mt)');

No solo disminuye la escorrentía total durante los meses de verano a medida que avanza la simulación, sino que el mes de máxima escorrentía también se cambia a principios del verano.

### Ejecuciones de proyección CMIP5
Ahora ha aprendido a simular y analizar un glaciar específico bajo un clima constante. Ahora daremos un paso más y simularemos dos glaciares diferentes, ubicados en diferentes regiones climáticas, forzados con las proyecciones climáticas de CMIP5.
Comenzamos inicializando los directorios de los glaciares:

In [ ]:
# We keep Hintereisferner from earlier, but also add a new glacier
rgi_ids = [rgi_id, 'RGI60-15.02420']

In [ ]:
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=5, prepro_border=80, prepro_base_url=base_url)

`gdirs` ahora contiene dos glaciares, uno en Europa Central y otro en los Himalayas Orientales:

In [ ]:
gdirs

Podemos echar un vistazo rápido al nuevo glaciar:

In [ ]:
sh = salem.transform_geopandas(gdirs[1].read_shapefile('outlines'))
(gv.Polygons(sh).opts(fill_color=None, color_index=None) *
 gts.tile_sources['EsriImagery'] * gts.tile_sources['StamenLabels']).opts(width=800, height=500, active_tools=['pan', 'wheel_zoom'])

#### Reducción climática
Antes de ejecutar nuestra simulación, tenemos que procesar los datos climáticos del glaciar i.i.e. reducir la escala: (Esto puede llevar algún tiempo)

In [ ]:
from oggm.shop import gcm_climate
bp = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/pr/pr_mon_CCSM4_{}_r1i1p1_g025.nc'
bt = 'https://cluster.klima.uni-bremen.de/~oggm/cmip5-ng/tas/tas_mon_CCSM4_{}_r1i1p1_g025.nc'
for rcp in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
    # Download the files
    ft = utils.file_downloader(bt.format(rcp))
    fp = utils.file_downloader(bp.format(rcp))
    workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs,
                                 # Name file to recognize it later
                                 filesuffix='_CCSM4_{}'.format(rcp),
                                 # temperature projections
                                 fpath_temp=ft,
                                 # precip projections
                                 fpath_precip=fp,
                                 );

#### Ejecución de proyección
Con la reducción de escala completa, podemos ejecutar las simulaciones forzadas:

In [ ]:
for rcp in ['rcp26', 'rcp45', 'rcp60', 'rcp85']:
    rid = '_CCSM4_{}'.format(rcp)
    workflow.execute_entity_task(tasks.run_with_hydro, gdirs,
                                 run_task=tasks.run_from_climate_data, ys=2020,
                                 # use gcm_data, not climate_historical
                                 climate_filename='gcm_data',
                                 # use the chosen scenario
                                 climate_input_filesuffix=rid,
                                 # this is important! Start from 2020 glacier
                                 init_model_filesuffix='_historical',
                                 # recognize the run for later
                                 output_filesuffix=rid,
                                 # add monthly diagnostics
                                 store_monthly_hydro=True);

In [ ]:
# Create the figure
f, ax = plt.subplots(figsize=(18, 7), sharex=True)
# Loop over all scenarios
for i, rcp in enumerate(['rcp26', 'rcp45', 'rcp60', 'rcp85']):
    file_id = f'_CCSM4_{rcp}'
    # Open the data, gdirs[0] correspond to Hintereisferner.
    with xr.open_dataset(gdirs[0].get_filepath('model_diagnostics', filesuffix=file_id)) as ds:
        # Load the data into a dataframe
        ds = ds.isel(time=slice(0, -1)).load()

    # Select annual variables
    sel_vars = [v for v in ds.variables if 'month_2d' not in ds[v].dims]
    # And create a dataframe
    df_annual = ds[sel_vars].to_dataframe()

    # Select the variables relevant for runoff.
    runoff_vars = ['melt_off_glacier', 'melt_on_glacier', 'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
    # Convert to mega tonnes instead of kg.
    df_runoff = df_annual[runoff_vars].clip(0) * 1e-9
    # Sum the variables each year "axis=1", take the 11 year rolling mean
    # and plot it.
    df_runoff.sum(axis=1).rolling(window=11).mean().plot(ax=ax, label=rcp, color=sns.color_palette("rocket")[i])

ax.set_ylabel('Annual runoff (Mt)'); ax.set_xlabel('Year'); plt.title(gdirs[1].rgi_id); plt.legend();

Para Hintereisferner, la escorrentía continúa disminuyendo a lo largo del siglo XXI para todos los escenarios, lo que indica que el **pico de agua** ya se alcanzó en algún momento del pasado. Este es el caso de muchos glaciares europeos. ¿Qué pasa con nuestro glaciar sin nombre en el Himalaya?

In [ ]:
# Create the figure
f, ax = plt.subplots(figsize=(18, 7), sharex=True)
# Loop over all scenarios
for i, rcp in enumerate(['rcp26', 'rcp45', 'rcp60', 'rcp85']):
    file_id = f'_CCSM4_{rcp}'
    # Open the data, gdirs[1] correspond to the unnamed glacier.
    with xr.open_dataset(gdirs[1].get_filepath('model_diagnostics', filesuffix=file_id)) as ds:
        # Load the data into a dataframe
        ds = ds.isel(time=slice(0, -1)).load()

    # Select annual variables
    sel_vars = [v for v in ds.variables if 'month_2d' not in ds[v].dims]
    # And create a dataframe
    df_annual = ds[sel_vars].to_dataframe()

    # Select the variables relevant for runoff.
    runoff_vars = ['melt_off_glacier', 'melt_on_glacier',
                   'liq_prcp_off_glacier', 'liq_prcp_on_glacier']
    # Convert to mega tonnes instead of kg.
    df_runoff = df_annual[runoff_vars].clip(0) * 1e-9
    # Sum the variables each year "axis=1", take the 11 year rolling mean
    # and plot it.
    df_runoff.sum(axis=1).rolling(window=11).mean().plot(ax=ax, label=rcp, color=sns.color_palette("rocket")[i])

ax.set_ylabel('Annual runoff (Mt)'); ax.set_xlabel('Year'); plt.title(rgi_id); plt.legend();

A diferencia de Hintereisferner, estas proyecciones indican que la escorrentía anual aumentará en todos los escenarios para la primera mitad del siglo. Los escenarios de RCP más altos pueden alcanzar el **pico de agua** más adelante en el siglo, ya que el exceso de derretimiento puede seguir aumentando. Por otro lado, para los escenarios de RCP más bajos, el glaciar podría estar acercándose a un nuevo equilibrio, lo que reduce la escorrentía a principios de siglo ([Rounce et. al., 2020](https://www.frontiersin.org/articles/10.3389/feart.2019.00331/full)). Después de alcanzar el **pico de agua** (RCP2.6: ~2055, RCP8.5: ~2070 en estas proyecciones), la escorrentía anual comienza a disminuir. Esta disminución se debe a que el glaciar que se encoge ya no puede soportar los altos niveles de derretimiento.

## ¿Otra ejecución de proyección con sesgo de temperatura o precipitación?
TAREAS: incluya algo sobre cómo el sesgo de temperatura afecta el tamaño del glaciar y la producción de agua de deshielo.

## Llévate puntos a casa

- Los glaciares en equilibrio *no* son recursos hídricos netos: ganan tanta masa como liberan durante un año.
- Sin embargo, tienen una función amortiguadora estacional: liberan agua durante los meses de deshielo.
- El tamaño de un glaciar influye en la disponibilidad de agua aguas abajo durante la estación seca. El impacto es más importante si la temporada de derretimiento (cálida) coincide con la estación seca (ver [Kaser et al., 2010](https://www.pnas.org/content/107/47/20223)).
- Cuando los glaciares se derriten, se convierten en recursos hídricos netos a lo largo del año. El "pico de agua" es el momento en el que el suministro de deshielo de los glaciares alcanza su máximo, es decir, cuando se produce la máxima escorrentía.

## Referencias

- Kaser, G., Großhauser, M. y Marzeion, B.: Potencial de contribución de los glaciares a la disponibilidad de agua en diferentes regímenes climáticos, PNAS, 07 (47) 20223-20227, [doi:10.1073/pnas.1008162107](https://doi.org/10.1073/pnas.1008162107), 2010
- Huss, M. y Hock, R.: Respuesta hidrológica a escala mundial a la futura pérdida de masa de los glaciares, Nat. Clima Chang., 8(2), 135–140, [doi:10.1038/s41558-017-0049-x](https://doi.org/10.1038/s41558-017-0049-x), 2018.
- Marzeion, B., Kaser, G., Maussion, F. y Champollion, N.: Influencia limitada de la mitigación del cambio climático en la pérdida de masa glaciar a corto plazo, Nat. Clima Chang., 8, [doi:10.1038/s41558-018-0093-1](https://doi.org/10.1038/s41558-018-0093-1), 2018.
- Rounce, D. R., Hock, R., McNabb, R. W., Millan, R., Sommer, C., Braun, M. H., Malz, P., Maussion, F., Mouginot, J., Seehaus, T. C. y Shean, D. E. : Las estimaciones del espesor de los escombros globales distribuidos revelan que los escombros tienen un impacto significativo en el balance de masa de los glaciares, Geophys. Res. Lett., [doi:10.1029/2020GL091311](https://doi.org/10.1029/2020GL091311), 2021.

## ¿Que sigue?

xx_markdown_enlace_xx